# LEAST COST PATH GENERATION SCRIPT
Allison Smith
June 03, 2023

This script reads in slope, vegetation density, and terrain roughness layers generated from 3DEP lidar data, preprocesses data, creates rate and pace rasters, and generates least cost paths using ArcGIS tools.

## Load packages and data

In [22]:
#import necessary libraries
print("Importing libraries...")
import os
import arcpy
from arcpy import env
from arcpy.sa import *
from time import ctime
arcpy.CheckOutExtension("spatial")
time = ctime()

Importing libraries...


In [23]:
#define directories used for lcp components
f"{time} setting up directories, processing extent..."
main_dir = "E:\\a_smith\\lakeMtnsSite\\"
slope_dir = "E:\\a_smith\\lakeMtnsSite\\slope\\slope.tif"
veg_dir = "E:\\a_smith\\lakeMtnsSite\\veg\\veg.tif"
terr_dir = "E:\\a_smith\\lakeMtnsSite\\terrain\\roughness.tif"
dtm_dir = "E:\\a_smith\\lakeMtnsSite\\dtm\\dtm.tif"

In [24]:
#load data including rasters(slope, vegetation density, terrain roughness) 
# and polygons(starting and stopping points, and study site)
f"{time} loading data..."
slope = Raster(slope_dir)
veg = Raster(veg_dir)
terr = Raster(terr_dir)
dtm = Raster(dtm_dir)
studySite = "E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\studySite.shp"
start1 = "E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\start1.shp"
start2 = "E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\start2.shp"
start3 = "E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\start3.shp"
safetyZone = r"E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\safetyZone.shp"

In [25]:
#define and create directory that new layers (distance accumulation and lcps) will save to 
working_dir = main_dir + "lcp_generation\\"
if not os.path.exists(working_dir):
    os.mkdir(working_dir)

In [26]:
#define environments
#workspace
env.workspace = working_dir
#cell size
arcpy.env.cellSize = 20
#raster grid origin/alignment
arcpy.env.snapRaster = dtm
#output coordinate system
arcpy.env.outputCoordinateSystem = dtm
#resampling method
arcpy.env.resamplingMethod = "BILINEAR"
#processing extent
arcpy.env.extent = dtm
#mask/bounding box
arcpy.env.mask = dtm

## Preprocessing layers

In this step we are trimming the slope, vegetation density, and terrain roughness layers down to the study site size and aggregating them to the correct resolution. 

In [27]:
#extract rasters to study site polygon
f"{time} trimming to study site..."
print(ctime() + " trimming to study site...")
slo_sub = ExtractByMask(slope, studySite, "INSIDE", studySite)
veg_sub = ExtractByMask(veg, studySite, "INSIDE", studySite)
terr_sub = ExtractByMask(terr, studySite, "INSIDE", studySite)
dtm_sub = ExtractByMask(dtm, studySite, "INSIDE", studySite)

Fri Jun  9 11:36:52 2023 trimming to study site...


In [28]:
#aggregate all rasters to cell size of the largest cell size
#in this case, vegetation has cell size 20m, so others need to be aggregated to 20m
print(ctime() + " aggregating to 20m cell size...")
slo_agg = Aggregate(slo_sub, 20, "MEAN", "EXPAND", "NODATA")
#veg does not get aggregated because it is the limiting cell size (20m)
#veg_agg = Aggregate(veg_sub, cell_factor, "MEAN", "EXPAND", "NODATA")
terr_agg = Aggregate(terr_sub, 4, "MEAN", "EXPAND", "NODATA")
dtm_agg = Aggregate(dtm_sub, 20, "MEAN", "EXPAND", "NODATA")

Fri Jun  9 11:37:14 2023 aggregating to 20m cell size...


## Raster Calculations

Here we are combining the slope, vegetation, and terrain layers to generate the travel rate and pace rasters for least cost path generation. 

In [12]:
#create travel rate raster (meters per second) based on Campbell et al (2017) travel rate equation
#invert travel rate raster to create travel pace (seconds per meter); how long it takes to traverse
# a pixel, given the slope/veg/terrain conditions
#the travel rate calculation is subject to change as the new equation becomes available
print(ctime() + " performing raster calculations...")
travel_rt =  1.662 + (-1.076 *  veg_sub) + (-9.011 *  terr_agg) + (-0.05191 *  slo_agg) + (-0.01127 * (slo_agg  * slo_agg))
travel_rt.save("travelRt.tif")
travel_pc = 1 / travel_rt
travel_pc.save("travelPc.tif")

Fri Jun  9 10:29:09 2023 performing raster calculations...


## LCP Operations

In the final step, we are creating the distance accumulation rasters and back direction rasters which feed the optimal path as line tool to generate our LCPs for each starting point. 

In [13]:
#creating and saving out back direction rasters
outback1 = "outback1.tif"
outback2 = "outback2.tif"
outback3 = "outback3.tif"

In [14]:
#creating distance accumulation & back direction rasters based on starting points & travel pace raster
print(ctime() + " creating distance accumulation rasters...")
dist_acc1 = DistanceAccumulation(in_source_data = start1, 
                                 in_surface_raster = dtm_agg, 
                                 in_cost_raster = travel_pc, 
                                 out_back_direction_raster = "outback1.tif")
dist_acc2 = DistanceAccumulation(in_source_data = start2, 
                                 in_surface_raster = dtm_agg, 
                                 in_cost_raster = travel_pc, 
                                 out_back_direction_raster = "outback2.tif")
dist_acc3 = DistanceAccumulation(in_source_data = start3, 
                                 in_surface_raster = dtm_agg, 
                                 in_cost_raster = travel_pc, 
                                 out_back_direction_raster = "outback3.tif")

Fri Jun  9 10:29:29 2023 creating distance accumulation rasters...


In [15]:
#creating LCPs based on distance accumulation & back direction rasters, as well as safety zone
print(ctime() + " generating LCPs...")
lcp1 = OptimalPathAsLine(in_destination_data = safetyZone, 
                         in_distance_accumulation_raster = dist_acc1, 
                         in_back_direction_raster = outback1, 
                         out_polyline_features = "/lcp1.shp",
                         path_type = "EACH_ZONE")
lcp2 = OptimalPathAsLine(in_destination_data = safetyZone, 
                         in_distance_accumulation_raster = dist_acc2, 
                         in_back_direction_raster = outback2, 
                         out_polyline_features = "/lcp2.shp",
                         path_type = "EACH_ZONE")
lcp3 = OptimalPathAsLine(in_destination_data = safetyZone, 
                         in_distance_accumulation_raster = dist_acc3, 
                         in_back_direction_raster = outback3, 
                         out_polyline_features = "/lcp3.shp",
                         path_type = "EACH_ZONE")

Fri Jun  9 10:29:43 2023 generating LCPs...


ExecuteError: ERROR 010511: Internal error in cost path tracing: hr: 0x80041098.

ERROR 010423: E:\a_smith\lakeMtnsSite\lcp_generation\outback1.tif does not have valid statistics as required by the operation.
Failed to execute (OptimalPathAsLine).
